<a href="https://colab.research.google.com/github/eugeniapramesti/cnn/blob/main/flowercnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz

--2023-11-07 21:34:11--  https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.207, 142.250.31.207, 142.251.111.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228813984 (218M) [application/x-compressed-tar]
Saving to: ‘flower_photos.tgz’

flower_photos.tgz   100%[===================>] 218.21M   253MB/s    in 0.9s    

2023-11-07 21:34:12 (253 MB/s) - ‘flower_photos.tgz’ saved [228813984/228813984]



In [ ]:
!tar -xzf flower_photos.tgz

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import load_img

In [ ]:
path= 'flower_photos/'
class_names=[]
for i in os.listdir(path):
  if i != 'LICENSE.txt':
    class_names.append(i)

class_names

['tulips', 'roses', 'sunflowers', 'dandelion', 'daisy']

In [ ]:
dataset=[]
labels=[]

for i in class_names:
  class_path= path+i+'/'
  for j in os.listdir(class_path):
    image_path= class_path+j
    image_data= load_img(image_path, color_mode= 'grayscale')
    image_data= image_data.resize((180, 180))
    image_data= np.array(image_data)
    dataset.append(image_data)
    labels.append(class_names.index(i))

In [ ]:
dataset= np.array(dataset)
labels= np.array(labels)

In [ ]:
dataset.shape

(3670, 180, 180)

In [ ]:
dataset= dataset.reshape(dataset.shape[0], dataset.shape[1], dataset.shape[2], 1).astype("float32")
dataset/=255

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x_train ,x_test, y_train,y_test = train_test_split(dataset, labels, train_size = 0.7, random_state=123)
x_test, x_val, y_test, y_val= train_test_split(x_test, y_test, train_size = 0.5, random_state=123)

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
flip=tf.keras.layers.RandomFlip(mode="horizontal_and_vertical")
rotation= tf.keras.layers.RandomRotation(0.2)
brightness= tf.keras.layers.RandomBrightness(0.3)

In [ ]:
x_train= flip(x_train)
x_train= rotation(x_train)
x_train= brightness(x_train)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
callback= EarlyStopping(monitor= "val_loss", patience=3, verbose= 1, mode= 'min')

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential

In [ ]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(180,180, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, kernel_size=(3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(128, kernel_size=(3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(rate=0.5),
    Dense(64, activation="relu"),
    Dropout(rate=0.5),
    Dense(5, activation="softmax")
])

In [ ]:
model.compile(optimizer= Adam(1e-3), loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=  5, batch_size=100, validation_data= (x_val, y_val), callbacks= callback)

Epoch 1/5
26/26 [==============================] - 162s 6s/step - loss: 2.5176 - accuracy: 0.2304 - val_loss: 1.6073 - val_accuracy: 0.2523
Epoch 2/5
26/26 [==============================] - 144s 6s/step - loss: 1.6119 - accuracy: 0.2367 - val_loss: 1.6063 - val_accuracy: 0.2523
Epoch 3/5
26/26 [==============================] - 151s 6s/step - loss: 1.6041 - accuracy: 0.2472 - val_loss: 1.6019 - val_accuracy: 0.2523
Epoch 4/5
26/26 [==============================] - 142s 5s/step - loss: 1.6342 - accuracy: 0.2301 - val_loss: 1.6024 - val_accuracy: 0.2523
Epoch 5/5
26/26 [==============================] - 143s 6s/step - loss: 1.6016 - accuracy: 0.2456 - val_loss: 1.6011 - val_accuracy: 0.2523


In [ ]:
eval= model.evaluate(x_val, y_val, batch_size= 100)

In [ ]:
predict= model.predict(x_test)
y_pred= np.argmax(predict, axis=1)